In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input,Flatten, Embedding, Bidirectional, LSTM, Dense, Concatenate, TimeDistributed

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense,Layer
class SelectHiddenState(Layer):
    def __init__(self, **kwargs):
        super(SelectHiddenState, self).__init__(**kwargs)

    def call(self, lstm_output, scalar_input):
        # Ensure scalar_input is an integer for indexing
        timestep_index = tf.cast(tf.squeeze(scalar_input, axis=-1), tf.int32)
        # Gather the specific hidden state for each batch
        selected_state = tf.gather(lstm_output, timestep_index, batch_dims=1, axis=1)
        return selected_state

# Example usage with your model
# lstm_output is from your BiLSTM layer
# scalar_input is your additional input
max_sentence_length = 100  # Maximum length of sentence embeddings
embedding_dim = 512        # Dimension of sentence embeddings
max_char_length = 15       # Maximum length of a word in characters
char_vocab_size = 28       # Number of unique characters
num_diacritics = 8         # Number of possible diacritics for each character, including no diacritic

# Parameters
lstm_units = 32

# Character input
char_input = Input(shape=(max_char_length, char_vocab_size))

# Inputs
sentence_input = Input(shape=(max_sentence_length, embedding_dim))
scalar_input = Input(shape=(1,), name='scalar_input')

# BiLSTM layer with return_state
bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True))
bi_lstm_output, forward_h, forward_c, backward_h, backward_c = bi_lstm(sentence_input)
print( forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)
#, forward_h, forward_c, backward_h, backward_c
# Average the forward and backward states (or choose another method to combine them)
select_state_layer = SelectHiddenState()
hidden_state_nth_timestep = select_state_layer(bi_lstm_output, scalar_input)

#hidden_state_nth_timestep = bi_lstm_output[:, scalar_input[1], :]
print(bi_lstm_output.shape)

# RNN layer with initial state from BiLSTM
rnn_cell = SimpleRNNCell(64)
rnn_layer = RNN(rnn_cell, return_sequences=True)
rnn_output = rnn_layer(char_input,initial_state=hidden_state_nth_timestep)

# Output layer
output_layer = Dense(num_diacritics, activation='softmax')(rnn_output)

# Build and compile the model
# Assuming sentence_input and scalar_input are defined as Input layers
model = Model(inputs=[sentence_input, char_input,scalar_input], outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()
